In [20]:
# Software Development Kits (SDKs)
import smartsheet

In [21]:
# Token
TK = 'hj8lpncr2xthejeb65h2bn24a3';

# Initialize client
smartsheet_client = smartsheet.Smartsheet(TK);

# Make sure we don't miss any errors
smartsheet_client.errors_as_exceptions(True)

In [22]:
# assume user
smartsheet_client.assume_user("wwei@briskheat.com")

In [23]:
import os
from datetime import date
from pprint import pprint

from simple_smartsheet import Smartsheet
from simple_smartsheet.models import Sheet, Column, Row, Cell, ColumnType

In [24]:
#TOKEN = os.getenv('HOME')
#print(TOKEN)
SHEET_NAME = 'Creating a test sheet'
smartsheet = Smartsheet(TK)

In [25]:
sheets = smartsheet.sheets.list()
pprint(sheets)

[Sheet(name='Creating a test sheet', id=2599143702062980),
 Sheet(name='UTR - Record20201022', id=6658876713002884)]


In [26]:
# Delete the test sheet if already exists
for sheet in sheets:
    if sheet.name == SHEET_NAME:
        smartsheet.sheets.delete(id = sheet.id)
    print(sheet.id)

2599143702062980
6658876713002884


In [27]:
# create a new Sheet
new_sheet_skeleton = Sheet(
    name = SHEET_NAME,
    columns = [
        Column(primary=True, title="Full Name", type=ColumnType.TEXT_NUMBER),
        Column(title = "Number of read books", type=ColumnType.TEXT_NUMBER),
        Column(title = "Birth date", type=ColumnType.DATE),
        Column(title = "Library member", type=ColumnType.CHECKBOX),
    ],
)

In [28]:
sheets = smartsheet.sheets.list()
pprint(sheets)

[Sheet(name='UTR - Record20201022', id=6658876713002884)]


In [29]:
# print the sheet object attributes used by the Smartsheet API (camelCase)
pprint(new_sheet_skeleton.dump())

{'columns': [{'primary': True, 'title': 'Full Name', 'type': 'TEXT_NUMBER'},
             {'title': 'Number of read books', 'type': 'TEXT_NUMBER'},
             {'title': 'Birth date', 'type': 'DATE'},
             {'title': 'Library member', 'type': 'CHECKBOX'}],
 'effectiveAttachmentOptions': [],
 'name': 'Creating a test sheet',
 'rows': []}


In [30]:
# add the sheet via API
result = smartsheet.sheets.create(new_sheet_skeleton)
sheet = result.obj
print(f"ID of the created sheet is {sheet.id!r}")

ID of the created sheet is 5778037976393604


In [31]:
# retrieve a sheet by name
# this object is exacly the same as result.obj
sheet = smartsheet.sheets.get(SHEET_NAME)

In [32]:
# get columns details by column title (case-sensitive)
full_name_column = sheet.get_column("Full Name")
pprint(full_name_column.__dict__)
num_books_column = sheet.get_column("Number of read books")
pprint(num_books_column.__dict__)

{'auto_number_format': None,
 'contact_options': None,
 'format': None,
 'hidden': None,
 'id': 2936367515756420,
 'index': 0,
 'locked': None,
 'locked_for_user': None,
 'options': None,
 'primary': True,
 'symbol': None,
 'system_column_type': None,
 'tags': None,
 'title': 'Full Name',
 'type': <ColumnType.TEXT_NUMBER: 'TEXT_NUMBER'>,
 'validation': False,
 'version': 0,
 'width': 150}
{'auto_number_format': None,
 'contact_options': None,
 'format': None,
 'hidden': None,
 'id': 7439967143126916,
 'index': 1,
 'locked': None,
 'locked_for_user': None,
 'options': None,
 'primary': None,
 'symbol': None,
 'system_column_type': None,
 'tags': None,
 'title': 'Number of read books',
 'type': <ColumnType.TEXT_NUMBER: 'TEXT_NUMBER'>,
 'validation': False,
 'version': 0,
 'width': 150}


In [33]:
# add rows (cells are created using different ways)
# second way is the easist
new_rows = [
    Row(
        to_top = True,
        cells = [
            Cell(column_id=full_name_column.id, value='William Wei'),
            Cell(column_id=num_books_column.id, value=5),
        ],
    ),
    Row(
        to_top = True,
        cells = sheet.make_cells(
            {"Full Name": "Bob Lee", "Number of read books": 2}
        )
    ),
    Row(
        to_top = True,
        cells = [
            sheet.make_cell("Full Name", "Charlie Brown"),
            sheet.make_cell("Number of read books", 1),
            sheet.make_cell("Birth date", date(1990, 1, 1)),
        ],
    ),
]
smartsheet.sheets.add_rows(sheet.id, new_rows);

In [35]:
# sort rows by columns "Full Name" descending / returns updated sheet
sheet = smartsheet.sheets.sort_rows(
    sheet, [{"column_title": "Full Name", "descending": False}]
)
print("\n Sheet after adding rows:")
# print a list of dictionaries containing column titles and values for each row
pprint(sheet.as_list())


 Sheet after adding rows:
[{'Birth date': None,
  'Full Name': 'Bob Lee',
  'Library member': False,
  'Number of read books': 2},
 {'Birth date': datetime.date(1990, 1, 1),
  'Full Name': 'Charlie Brown',
  'Library member': False,
  'Number of read books': 1},
 {'Birth date': None,
  'Full Name': 'William Wei',
  'Library member': False,
  'Number of read books': 5}]


In [37]:
# get a specific cell and updating it:
row_id_to_delete = None
rows_to_update = []

for row in sheet.rows:
    full_name = row.get_cell("Full Name").value
    num_books = row.get_cell("Number of read books").value
    #print(f"{full_name} has read {num_books} books")
    if full_name.startswith("Charlie"):
        updated_row = Row(
            id = row.id, cells = [sheet.make_cell("Number of read books", 15)]        
        )
        rows_to_update.append(updated_row)
        print(full_name)
    elif full_name.startswith("Bob"):
        row_id_to_delete = row.id #used later
        
# update rows
smartsheet.sheets.update_rows(sheet.id, rows_to_update);


Charlie Brown


In [ ]:
# get an updated sheet
sheet = smartsheet.sheets.get(id=sheet.id)
print("\nSheet after updateing rows:")
pprint(sheet.as_list())

In [ ]:
# delete a row
smartsheet.sheets.delete_row(sheet.id, row_id_to_delete)

# get an updated sheet
sheet = smartsheet.sheets.get(id=sheet.id)
print("\nSheet after updateing rows:")
pprint(sheet.as_list())

In [ ]:
# delete a sheet by name
smartsheet.sheets.delete(SHEET_NAME)
sheets = smartsheet.sheets.list()
pprint(sheets)